# TODO
* Understand the difference between this notebook copies and also the fit_distribution notebook
* Comment and refactor the code

In [ ]:

import sys 
sys.path.append(r'..\bin')
import analytics_stat.distributions as di

import warnings
warnings.filterwarnings('ignore')

import datetime
import scipy.stats as s
#import scipy.optimize as opt
#import scipy.special as special
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
%matplotlib inline
# %matplotlib notebook

import os
import json
import pandas as pd
import qgrid
from IPython.display import HTML, Image, display, clear_output
# display(HTML("<style>.container { width:95% !important; }</style>"))

# py.sign_in('diogo.antunes.goncalves', 'kt7T5JadNn2AhWk3DdPR')

import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)

display(HTML('''<script>
code_show=false;
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
To hide/show code, click <a href="javascript:code_toggle()">here</a>.''')) 



In [ ]:
import traitlets
from tkinter import Tk, filedialog


class Plotting:
    
    def __init__(self):
        pass
    
    def plot_line(self, json, x, y, name):
        x = json.x
        y = json.y
        name = json.name
        trace_line = go.Scatter(x = x, y = y, mode = 'lines', opacity = 0.8, name = name, xaxis = 'x', yaxis = 'y')
        
    def plot_scatter(self,  x, y, name):
        x = json.x
        y = json.y
        name = json.name
        trace_scatter = go.Scatter(x = x, y = y, mode = 'markers', opacity = 0.8, name = fn, xaxis = 'x', yaxis = 'y')
        
    def plot_histogram(self,  x, name, normed):
        x= json.x
        normed = json.type  # 'probability'
        trace_histogram = go.Histogram(x=x, histnorm=normed)
                           
    def create_layout(self, title, xlabel, ylabel):
        title = json.title
        xlabel = json.xlabel
        ylabel = json.ylabel
        layout = Layout(title=title, yaxis=dict(title=ylabel)) 
        
    def draw(self,trace,layout):
        fig = go.Figure(data = [trace_histogram],layout=layout)
        py.iplot(fig)


class StatPlot(Plotting):
    
    def __init__(self):
        
        pass
    
    def pdf(self, json):
        
        pdf_mle = dist_funs[dist_index[0]].pdf(params_dist_mle,  x_plot)



class Display:
    
    def __init__(self):
        
        # Select file button
        self._bsf = widgets.Button(layout=widgets.Layout(width='500px',height='50px'))
        self._bsf.add_traits(files=traitlets.traitlets.List())
        self._bsf.description = "Select File"
        self._bsf.icon = "square-o"
        self._bsf.on_click(self.select_json)
        
        
        # Load Json button
        self._blj = widgets.Button(layout=widgets.Layout(width='500px',height='50px'))
        self._blj.description = "Load File"
        self._blj.icon = 'download'
        self._blj.on_click(self.load_json)
        self._blj.disabled = True
        
        # Rank distributions button
        self._brd = widgets.Button(layout=widgets.Layout(width='500px',height='50px'))
        self._brd.description = 'Rank Distributions'
        self._brd.icon = "area-chart"
        self._brd.on_click(self.rank_distributions)
        self._brd.disabled = True
        
        # Select distributions
        self._sdist = widgets.SelectMultiple(
                    options=['weibull3p', 'weibull2p', 'normal', 'lognormal',
                  'exponential1p', 'exponential2p', 'logistic', 'loglogistic'],
                    value=['weibull3p', 'weibull2p', 'normal', 'lognormal',
                  'exponential1p', 'exponential2p', 'logistic', 'loglogistic'],
                    description='Dist. functions:',
                    disabled=True,
                    
                    )
        
        # Save model button
        self._bsm = widgets.Button(layout=widgets.Layout(width='500px',height='50px'))
        self._bsm.description = 'Save Best Model'
        self._bsm.icon = "floppy-o"
        self._bsm.on_click(self.save_model_json)
        self._bsm.disable = True
        
        # define layout
        box_layout = widgets.Layout(display='center',
                            flex_flow='row',
                            align_items='center',
                            border='None',
                            justify_content='space-between',
                            width='800px')
        
        self._box = widgets.Box(children=[self._bsf, self._blj, self._sdist, self._brd, self._bsm], layout=box_layout)
        display(self._box)
        
    def select_json(self,b): 
        # Create Tk root
        root = Tk()
        # Hide the main window
        root.withdraw()
        # Raise the root to the top of all windows.
        root.call('wm', 'attributes', '.', '-topmost', True)
        
        new_file = filedialog.askopenfilename(multiple=True,filetypes = [("JSON",'*.json')])
        if new_file != "":
            self._bsf.files = new_file

        print('Selected file: {}'.format(self._bsf.files ))

        self._bsf.description = "Files Selected"
        self._bsf.icon = "check-square-o"
        self._bsf.button_style = "success"
        
        self._blj.disabled = False

        
        
    def load_json(self, b):
        
        file_path = self._bsf.files
        if (type(file_path) == list) and (len(file_path)>0):
            with open(file_path[0], 'r') as f:
                self._json_file = json.load(f)
                
            self._ttf = np.array(self._json_file['ttf'])
            self._client = self._json_file['client_name']
            self._clientid = self._json_file['client_id']
            self._symptom = self._json_file['symptom']
                
            self._sdist.disabled = False
            self._brd.disabled = False
            self._blj.button_style = "success"
#             print('JSON loaded.')
        else:
            print('Select file.')
            
            
    def save_model_json(self, b):
        json_model = dict()
        json_model['version'] = 0.1 
        json_model['saved_time'] = str(datetime.datetime.now())
        
        if self._symptom is None:
            json_model['symptom'] = None
        elif isinstance(self._symptom, int):
            json_model['symptom'] = int(self._symptom)
        elif isinstance(self._symptom, list):
            json_model['symptom'] = self._symptom
        else:
            raise TypeError('Undefined type for Symptom')
            
        json_model['client_id'] = int(self._clientid)
        json_model['client_name'] = self._client
        
        json_model['input_json'] = os.path.splitext(os.path.basename(d._bsf.files[0]))[0]
        json_model['competing_models'] = list(self._sdist.value)
        
        json_model['best_model'] = dict()
        json_model['best_model']['law'] = self._combs[0].iloc[0].name
        json_model['best_model']['MLE'] = dict()
        json_model['best_model']['MLE']['params'] = self._res_mle[json_model['best_model']['law']]['params']
        json_model['best_model']['MLE']['goodness_of_fit'] = self._res_mle[json_model['best_model']['law']]['goodness_of_fit']
        
        json_model['best_model']['LSQ'] = dict()
        json_model['best_model']['LSQ']['params'] = self._res_ls[json_model['best_model']['law']]['params']
        json_model['best_model']['LSQ']['goodness_of_fit'] = self._res_ls[json_model['best_model']['law']]['goodness_of_fit']
                                
        with open("../models/stat/model_{}_{}.json".format(json_model['input_json'],
                                                           json_model['best_model']['law']), 'w') as f:
            json.dump(json_model, f)
        
        self._bsm.button_style = 'success'
        
        
    def rank_distributions(self, b):
        #print('Ranking from Maximum Likelihood Estimation fitting:')
        self._res_mle = di.rank_dist(self._ttf, method='MLE',
                                     distributions=list(self._sdist.value))

#         print('')
#         print(self._res_mle)
#         print('')

        #print('Ranking from Least Squares fitting:')
        self._res_ls = di.rank_dist(self._ttf, method='lsq', 
                                    distributions=list(self._sdist.value))
#         print(self._res_ls)
#         print('')
        #print('Ranking from the combination of both methods:')
        self._combs = di.combine_ranks(res_mle=self._res_mle, 
                                       res_ls=self._res_ls, display=True)
        
        di.plot_best(self._combs[0], self._ttf)
        
        self._brd.button_style = "success"
        self._bsm.disabled = False
    
#         print(self._combs)

# Fitting distributions to reliability data (Life data)

The database is one vector of "Time to Failure" values, given in periods of either seconds, minutes, hours, days, weeks, months, years. The found data-sets comprise hours of functioning which are $\Delta t$ between system failure and previous repair. 

MTTF – Mean Time To Failure (TTF – time to fail) can be interpreted as the difference between the time of a failure and the time of putting the system at work after a repairing event, as depicted in the figure bellow.



In [ ]:
display(Image('../imgs/ttf_example.png', width=500))

In [ ]:
d = Display()

In [ ]:
d._ttf